In [1]:
import pandas as pd
import numpy as np
import os
import shutil
import plotly.express as px
import plotly.graph_objects as go
import importlib
import performance_shr
importlib.reload(performance_shr)
from performance_shr import convert_to_float

In [2]:
default_threshold = 0.10   # eg 0.1 = 10%
file_dir = 'D:/Astro/Images/2025-05-07/eta-Aquariids/Lights'
file_name = 'ImageMetaData.csv'
file_path = os.path.join(file_dir, file_name)
converters_dict = {
    'FilePath': str,   
    'Gain': convert_to_float,   
    'DetectedStars': convert_to_float,   
    'HFR': convert_to_float,  
    'HFRStDev': convert_to_float,  
    'FWHM': convert_to_float,  
    'Eccentricity': convert_to_float,  
    'FocuserPosition': convert_to_float,  
    'Airmass': convert_to_float,  
    'MountRA': convert_to_float,  
    'MountDec': convert_to_float,  
    }

In [3]:
data = pd.read_csv(file_path, converters=converters_dict)
data['FileName'] = data['FilePath'].apply(lambda x: os.path.basename(x))
data = data[data['FilePath'].str.strip() != '' ]
data

,ExposureNumber,FilePath,FilterName,ExposureStart,Duration,Binning,CameraTemp,CameraTargetTemp,Gain,Offset,...,GuidingRMSDECArcSec,FocuserPosition,FocuserTemp,RotatorPosition,PierSide,Airmass,ExposureStartUTC,MountRA,MountDec,FileName
0,0,D:/Astro/Images/2025-05-07/eta-Aquariids/LIGHT...,NoFilter,2025-05-08 03:46,30,1x1,NaN,NaN,400.0,-1,...,0,204.0,NaN,0,NaN,2.1321,2025-05-07T17:46:18.0029455Z,339.445269,-0.419564,2025-05-08_03-46-18_NoFilter_44c_30.00s_0000.cr3
1,1,D:/Astro/Images/2025-05-07/eta-Aquariids/LIGHT...,NoFilter,2025-05-08 03:46,30,1x1,NaN,NaN,400.0,-1,...,0,204.0,NaN,0,NaN,2.1244,2025-05-07T17:46:52.5243651Z,339.445368,-0.419413,2025-05-08_03-46-52_NoFilter_41c_30.00s_0001.cr3
2,2,D:/Astro/Images/2025-05-07/eta-Aquariids/LIGHT...,NoFilter,2025-05-08 03:47,30,1x1,NaN,NaN,400.0,-1,...,0,204.0,NaN,0,NaN,2.1168,2025-05-07T17:47:27.0949320Z,339.446124,-0.419504,2025-05-08_03-47-27_NoFilter_39c_30.00s_0002.cr3
3,3,D:/Astro/Images/2025-05-07/eta-Aquariids/LIGHT...,NoFilter,2025-05-08 03:48,30,1x1,NaN,NaN,400.0,-1,...,0,204.0,NaN,0,NaN,2.1088,2025-05-07T17:48:01.6548747Z,339.446410,-0.419460,2025-05-08_03-48-01_NoFilter_38c_30.00s_0003.cr3
4,4,D:/Astro/Images/2025-05-07/eta-Aquariids/LIGHT...,NoFilter,2025-05-08 03:48,30,1x1,NaN,NaN,400.0,-1,...,0,204.0,NaN,0,NaN,2.1014,2025-05-07T17:48:36.3188656Z,339.446719,-0.419587,2025-05-08_03-48-36_NoFilter_37c_30.00s_0004.cr3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,295,D:/Astro/Images/2025-05-07/eta-Aquariids/LIGHT...,NoFilter,2025-05-08 08:04,30,1x1,NaN,NaN,400.0,-1,...,0,203.0,NaN,0,NaN,1.2091,2025-05-07T22:04:19.4766716Z,339.703708,-0.427759,2025-05-08_08-04-19_NoFilter_37c_30.00s_0295.cr3
296,296,D:/Astro/Images/2025-05-07/eta-Aquariids/LIGHT...,NoFilter,2025-05-08 08:05,30,1x1,NaN,NaN,400.0,-1,...,0,203.0,NaN,0,NaN,1.2103,2025-05-07T22:05:51.4751142Z,339.705121,-0.427744,2025-05-08_08-05-51_NoFilter_38c_30.00s_0296.cr3
297,297,D:/Astro/Images/2025-05-07/eta-Aquariids/LIGHT...,NoFilter,2025-05-08 08:07,30,1x1,NaN,NaN,400.0,-1,...,0,203.0,NaN,0,NaN,1.2116,2025-05-07T22:07:21.4651632Z,339.706839,-0.427718,2025-05-08_08-07-21_NoFilter_38c_30.00s_0297.cr3
298,298,D:/Astro/Images/2025-05-07/eta-Aquariids/LIGHT...,NoFilter,2025-05-08 08:08,30,1x1,NaN,NaN,400.0,-1,...,0,203.0,NaN,0,NaN,1.2128,2025-05-07T22:08:49.4369413Z,339.708334,-0.428162,2025-05-08_08-08-49_NoFilter_38c_30.00s_0298.cr3


In [4]:
def filter_percentiles(df, col='HFR', topq=False, bottomq=False, top=False, bottom=False, zero=True):
    top_threshold = top if top else df[col].quantile(1 - topq) if topq else False
    bottom_threshold = bottom if bottom else df[col].quantile(bottomq) if bottomq else False
    is_na = df[col].isna()
    is_false = is_na != is_na
    is_zero = df[col] == 0 if zero else is_false
    is_top = df[col] >= top_threshold if top_threshold else is_false
    is_bottom = df[col] <= bottom_threshold if bottom_threshold else is_false
    is_filtered = is_zero | is_na | is_top | is_bottom

    fig=go.Figure()
    color_map = {True: 'red', False: 'blue'}
    fig.add_trace(go.Scatter(x=df.ExposureNumber, y=df[col], name=col, mode='markers', marker=dict(color=np.vectorize(color_map.get)(is_filtered))))
    fig.update_layout(title=f'{col} vs Exposure No.', xaxis_title='Exposure No.', yaxis_title=col)
    fig.show()    

    print(f'{col}: TopThreshold={top_threshold:.3f} n({is_top.sum()}), BottomThreshold={bottom_threshold:.3f} n({is_bottom.sum()}), na=n({is_na.sum()}), zero=n({is_zero.sum()}), filtered=n({is_filtered.sum()})\n\n')
    return is_filtered

In [8]:
print(f'**** Rejecting Images based on top and bottom percentiles for HFR, FWHM, Eccentricity and #Stars Dectected ****')
is_badHFR = filter_percentiles(data, col='HFR', topq=0.1)
is_badHFRStDev = filter_percentiles(data, col='HFRStDev', topq=0.1, bottom=0.17)
is_badFWHM = filter_percentiles(data, col='FWHM', topq=.08, bottomq=0.01)
is_badEccentricity = filter_percentiles(data, col='Eccentricity', topq=0.1, bottomq=0.1)
is_badDetectedStars = filter_percentiles(data, col='DetectedStars', bottom=60)
is_rejected = is_badHFR | is_badHFRStDev | is_badFWHM |  is_badFWHM | is_badEccentricity | is_badDetectedStars
print(f'\nTotal Rejected: n({is_rejected.sum()}/{len(data)}, Total Images: n({len(data) - is_rejected.sum()})')


**** Rejecting Images based on top and bottom percentiles for HFR, FWHM, Eccentricity and #Stars Dectected ****


HFR: TopThreshold=1.974 n(30), BottomThreshold=0.000 n(0), na=n(0), zero=n(108), filtered=n(138)




HFRStDev: TopThreshold=0.217 n(30), BottomThreshold=0.170 n(118), na=n(0), zero=n(108), filtered=n(148)




FWHM: TopThreshold=108.267 n(16), BottomThreshold=98.294 n(2), na=n(110), zero=n(0), filtered=n(128)




Eccentricity: TopThreshold=0.376 n(19), BottomThreshold=0.290 n(19), na=n(110), zero=n(9), filtered=n(148)




DetectedStars: TopThreshold=0.000 n(0), BottomThreshold=60.000 n(123), na=n(0), zero=n(107), filtered=n(123)



Total Rejected: n(174/300, Total Images: n(126)


In [9]:
# Create the 'rejected' directory at the same level as file_dir if it doesn't exist 
parent_dir = os.path.dirname(file_dir) 
rejected_dir = os.path.join(parent_dir, 'Rejected') 
os.makedirs(rejected_dir, exist_ok=True)

In [10]:
rejects = data[is_rejected].FileName
for filename in rejects:
    src_path = os.path.join(file_dir, filename)
    dest_path = os.path.join(rejected_dir, filename)
    if os.path.exists(src_path): 
        shutil.move(src_path, dest_path) 
        print(f'Moved: {filename}')

print('File moving complete.')

Moved: 2025-05-08_03-47-27_NoFilter_39c_30.00s_0002.cr3
Moved: 2025-05-08_03-56-05_NoFilter_31c_30.00s_0017.cr3
Moved: 2025-05-08_03-56-39_NoFilter_30c_30.00s_0018.cr3
Moved: 2025-05-08_03-59-32_NoFilter_29c_30.00s_0023.cr3
Moved: 2025-05-08_04-02-25_NoFilter_28c_30.00s_0028.cr3
Moved: 2025-05-08_04-03-34_NoFilter_28c_30.00s_0030.cr3
Moved: 2025-05-08_04-09-54_NoFilter_27c_30.00s_0041.cr3
Moved: 2025-05-08_04-10-28_NoFilter_27c_30.00s_0042.cr3
Moved: 2025-05-08_04-23-50_NoFilter_28c_30.00s_0061.cr3
Moved: 2025-05-08_04-24-24_NoFilter_27c_30.00s_0062.cr3
Moved: 2025-05-08_04-26-07_NoFilter_27c_30.00s_0065.cr3
Moved: 2025-05-08_04-28-25_NoFilter_26c_30.00s_0069.cr3
Moved: 2025-05-08_04-30-44_NoFilter_26c_30.00s_0073.cr3
Moved: 2025-05-08_04-32-27_NoFilter_26c_30.00s_0076.cr3
Moved: 2025-05-08_04-33-37_NoFilter_26c_30.00s_0078.cr3
Moved: 2025-05-08_04-34-11_NoFilter_26c_30.00s_0079.cr3
Moved: 2025-05-08_04-34-46_NoFilter_26c_30.00s_0080.cr3
Moved: 2025-05-08_04-36-30_NoFilter_26c_30.00s_0